In [15]:
# csce478    Programming Assignment 4     Fall 2020                                   Due: Nov 17 11am
# Linear Support Vector Machine & Principle Component Analysis
# Contributers: Chungsum Kim, Devn Steiner, Jesse Reyes Cortes

#imported packages
import pandas as pd
import numpy as np
import math
import copy
import itertools
from random import sample 
import nltk
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from random import seed
seed(1)
from random import random

# helper functions
def accuracy(x,y):
    count = 0
    for i in range(0, len(x)):
        if(x[i]==0 and 0 == y[i]):
            count = count + 1
        elif(x[i]==1 and 0 == y[i]):
            count = count + 1
        elif(x[i]==2 and y[i]==2):
            count = count + 1
    
    return count/len(x)


def confusion_matrix(x,y):
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in range(0, len(y)):
        if(x[i]==0 and y[i] <= .5):
            TN = TN + 1
        elif(x[i]==0 and .5 < y[i]):
            FP = FP + 1
        elif(x[i]==1 and y[i] <= .5):
            FN = FN + 1
        elif(x[i]==1 and .5 < y[i]):
            TP = TP + 1

    return [[TN, FP],[FN, TP]]


def adaptive_learning_rate(t_0, t_1, iteration):
    return t_0/(iteration + t_1)


def kFoldHelper(X,Y, k):
    
    kData = []
    
    x = np.array_split(X,k)
    y = np.array_split(Y,k)
    
    for i in range(0, k):
        x_train = np.array([])
        y_train = np.array([])
        x_test = x[i]
        y_test = y[i]
        for j in range(0,k):
            if i != j:
                if(len(x_train)==0):
                    x_train = x[j]
                else:
                    x_train = np.concatenate((x_train,x[j]))
                y_train = np.concatenate((y_train, y[j]))
            
        kData.append([x_train,y_train,x_test,y_test])
    
    return kData


In [41]:
# Part A : Linear Support Vector Machine (SVM)

# A.1 implement Linear_SVC model class for performing binary classification

class Linear_SVC:
    def __init__(self, C=1, max_iter=100, tol=None, learning_rate="constant",learning_rate_init=0.001, t_0=1, t_1=1000, early_stopping=False, validation_fraction=0.1, **kwargs):
        self.C = C
        self.max_iter = max_iter
        self.tol = tol
        self.learning_rate = learning_rate
        self.learning_rate_init = learning_rate_init
        self.t_0 = t_0
        self.t_1 = t_1
        self.early_stopping = early_stopping
        self.validation_fraction = validation_fraction
        self.cost = np.array([])
        
        
    def fit(self, X, Y):
        self.X = X
        self.Y = Y
        
        # use to find labels for support vectors
        t = self.Y * 2 - 1
        
        # calculate the weight vector w with length of one sample with random small number
        self.w = np.array([])
        for i in range(0, len(self.X[0])):
            self.w = np.append(self.w, [random()])
        
        # intercept bias b originally set to 0
        self.b = 0
        
        prim_form = 1000000000000000
        
        # iterate 0 - max_iter unless stopped early
        for i in range(0, self.max_iter):
            
            # determine which are the support vectors & their labels
            svs = (t * (self.X.dot(self.w) + self.b) < 1).ravel()
            x_sv = self.X[svs]
            t_sv = t[svs]
            
#             # compute primary form
            prim_form = (1/2) * np.matmul(self.w, self.w.reshape(-1,1))
#             if prim_form > (1/2) * np.matmul(self.w, self.w.reshape(-1,1)):
#                 prim_form = (1/2) * np.matmul(self.w, self.w.reshape(-1,1))


            
            j = abs(prim_form + self.C * (np.sum(1-x_sv.dot(self.w.reshape(-1,1))) - self.b * np.sum(t_sv)))
            
            self.cost = np.append(self.cost, [j])
            
            # early stopping
            if j < self.tol:
                break
                
            #compute gradients
            grad_w = self.w - self.C * np.sum(x_sv)
            grad_b = -self.C * np.sum(t_sv)
            
            # determine the correct learning rate
            lr = self.learning_rate_init
            
            if self.learning_rate == "adaptive":
                lr = self.t_0/(i + self.t_1)
            
            #update gradients
            self.w = self.w - lr * grad_w
            self.b = self.b - lr * grad_b
            
        print("Costs j: ", self.cost)
        
        self.intercept = np.array([self.b]) 
        self.coef = np.array([self.w]) 
        self.support_vectors = np.array([x_sv])
        
        return self
            
        
    def predict(self, X):
        
        y_hat = np.array([])
        
        for i in range(0, len(X)):
            if self.w.dot(X[i]) + self.b >= 0:
                y_hat = np.append(y_hat, [1])
            else:
                y_hat = np.append(y_hat, [0])
        
        
        return y_hat
        
        
        
        
        
        

In [42]:
# Binary Classification using Linear_SVC Classifier

# A.2 read the iris data, X features: petal length & petal width, recode binary target such that Iris-Virginica is 1 or 0
data = load_iris()

pl = data["data"][:,2]
pw = data["data"][:,3]
X = np.zeros((len(pl), 2))
X[:,0] = pl
X[:,1] = pw


Y = np.where(data['target'] == 2, 1, 0)

In [43]:
# A.3 partition data into train & test set (80% - 20%)
def partition(X, Y, t):     
    test_size = math.ceil(len(Y)*t)
    training_size = len(Y)-test_size
    training_data = X[0:training_size][:]
    testing_data = X[training_size+1:len(Y)] [:]
    training_vector = Y[0:training_size]
    testing_vector = Y[training_size+1:len(Y)]
    
    return training_data, testing_data, training_vector, testing_vector

In [44]:
# A.4 Hyper-parameter training(kFold - C, learning_rate, max_iter, tol)

#testing params
Cs = [1,1.5]
learning_rate_inits = [.001, .1]
max_iters = [100, 50]
tol = [.0001, .01]
learning_rates = ["constant", "adaptive"]

data = kFoldHelper(X, Y, 5)

dict_svm = {"C":[], "learn rate init":[], "max iter":[], "tol":[], "learn rate":[], "accuracy":[], "confusion matrix":[]}

for c in Cs:
    for lri in learning_rate_inits:
        for mi in max_iters:
            for t in tol:
                for lr in learning_rates:
                    y_hat = np.array([])
                    for d in data:
                        
                        x_train = d[0]
                        y_train = d[1]
                        x_test = d[2]
                        y_test = d[3]
                                                
                        model = Linear_SVC(C=c,max_iter=mi,tol=t,learning_rate=lr,learning_rate_init=lri,early_stopping=True)
                        
                        model.fit(x_train,y_train)
                        
                        y_hat = np.append(y_hat, model.predict(x_test))
                        
                        
                    #update dict
                    dict_svm["C"].append(c)
                    dict_svm["learn rate init"].append(lri)
                    dict_svm["max iter"].append(mi)
                    dict_svm["tol"].append(t)
                    dict_svm["learn rate"].append(lr)
                    dict_svm["accuracy"].append(accuracy(Y,y_hat))
                    dict_svm["confusion matrix"].append(confusion_matrix(Y,y_hat))

Costs j:  [ 117.28160114  219.53119978  321.4884726   423.15410168  524.52876765
  625.61314966  726.40792543  826.91377121  927.13136181 1027.06137059
 1126.70446947 1226.06132893 1325.132618   1423.9190043  1522.421154
 1620.63973184 1718.57540115 1816.22882382 1913.60066034 2010.69156976
 2107.50220975 2204.03323652 2300.28530493 2396.25906838 2491.95517891
 2587.37428714 2682.5170423  2777.38409222 2871.97608335 2966.29366075
 3060.33746808 3154.10814765 3247.60634035 3340.83268573 3433.78782194
 3526.47238579 3618.88701269 3711.03233671 3802.90899054 3894.51760552
 3985.85881163 4076.9332375  4167.74151041 4258.28425629 4348.56209972
 4438.57566395 4528.32557088 4617.81244108 4707.03689377 4795.99954687
 4884.70101693 4973.14191921 5061.32286764 5149.24447481 5236.90735201
 5324.31210921 5411.45935507 5498.34969695 5584.98374087 5671.36209159
 5757.48535254 5843.35412585 5928.96901238 6014.33061167 6099.43952198
 6184.29634028 6268.90166226 6353.25608233 6437.3601936  6521.2145879

Costs j:  [ 217.64059101  322.12522843  426.21619888  529.91525019  633.22412094
  736.1445405   838.67822911  940.82689793 1042.5922491  1143.97597577
 1244.97976218 1345.60528373 1445.85420698 1545.72818977 1645.22888123
 1744.35792183 1843.11694348 1941.50756954 2039.53141486 2137.1900859
 2234.4851807  2331.41828899 2427.99099222 2524.2048636  2620.06146817
 2715.56236283 2810.70909642 2905.50320973 2999.94623557 3094.03969883
 3187.78511651 3281.18399776 3374.23784396 3466.94814873 3559.31639801
 3651.34407007 3743.0326356  3834.38355772 3925.39829204 4016.07828669
 4106.42498241 4196.43981254 4286.12420308 4375.47957277 4464.50733307
 4553.20888826 4641.58563546 4729.63896466 4817.3702588  4904.78089376
 4991.87223846 5078.64565485 5165.10249798 5251.24411603 5337.07185037
 5422.58703557 5507.79099946 5592.68506317 5677.27054116 5761.54874126
 5845.52096474 5929.18850628 6012.5526541  6095.6146899  6178.37588899
 6260.83752026 6343.00084625 6424.86712318 6506.437601   6587.713523

Costs j:  [ 170.39025164  272.62174624  374.45912315  475.90413999  576.95854506
  677.6240774   777.90246684  877.79543408  977.30469069 1076.43193925
 1175.17887334 1273.5471776  1371.53852784 1469.15459102 1566.39702536
 1663.26748037 1759.7675969  1855.89900723 1951.66333506 2047.0621956
 2142.09719565 2236.76993359 2331.08199946 2425.03497503 2518.63043383
 2611.8699412  2704.75505433 2797.28732235 2889.46828633 2981.29947935
 3072.78242657 3163.91864525 3254.70964478 3345.15692678 3435.26198512
 3525.02630596 3614.4513678  3703.53864153 3792.28959048 3880.70567046
 3968.7883298  4056.53900942 4143.95914283 4231.05015622 4317.81346848
 4404.25049123 4490.36262891 4576.15127877 4661.61783095 4746.76366851
 4831.59016747 4916.09869684 5000.2906187  5084.1672882  5167.73005362
 5250.98025641 5333.91923124 5416.548306   5498.86880191 5580.88203348
 5662.58930861 5743.9919286  5825.09118821 5905.88837567 5986.38477274
 6066.58165473 6146.48029056 6226.08194279 6305.38786766 6384.399315

Costs j:  [   90.67286827 13012.81860996 22473.91930206 29251.15723987
 33962.09170897 37096.18319448 39040.20840672 40098.73882671
 40510.63146518 40462.29920078 40098.38129522 39530.31589746
 38843.22022044 38101.40628771 37352.79721128 36632.45804754
 35965.41409428 35368.89618456 34854.12560086 34427.72945894
 34092.85981175 33850.07550253 33698.03430656 33634.03362287
 33654.43048563 33754.96561935 33931.01138507 34177.75953196
 34490.36149994 34864.03146609 35294.12027356 35776.16672901
 36305.93142718 36879.41719503 37492.87939309 38142.82862701
 38826.02787462 39539.48559651 40280.44604943 41046.37774468
 41834.96077336 42644.07354658 43471.77936086 44316.31309163
 43431.26372106 44350.85276455 45274.50097473 46201.80660143
 47132.40727787 48065.97623271]
Costs j:  [ 124.24990298  226.59953252  328.55466464  430.11705851  531.28846401
  632.07062171  732.46526301  832.47411011  932.09887614 1031.34126519
 1130.20297233 1228.68568374 1326.7910767  1424.52081967 1521.87657235
 16

Costs j:  [ 155.36171628  251.57834165  347.44317732  442.95777571  538.12368106
  632.94242943  727.41554881  821.54455911  915.33097226 1008.77629224
 1101.88201511 1194.64962909 1287.08061461 1379.17644431 1470.93858314
 1562.3684884  1653.46760975 1744.2373893  1834.67926164 1924.79465387
 2014.58498567 2104.05166934 2193.19610983 2282.0197048  2370.52384467
 2458.70991265 2546.57928477 2634.13332997 2721.37341009 2808.30087996
 2894.9170874  2981.2233733  3067.22107163 3152.91150951 3238.29600722
 3323.37587829 3408.15242948 3492.62696087 3576.80076586 3660.67513125
 3744.25133725 3827.53065755 3910.51435931 3993.20370324 4075.59994363
 4157.7043284  4239.51809909 4321.04249096 4402.27873299 4483.22804792
 4563.89165229 4644.27075651 4724.36656482 4804.18027542 4883.71308043
 4962.96616596 5041.94071215 5120.63789318 5199.05887735 5277.20482705
 5355.07689885 5432.67624352 5510.00400605 5587.06132569 5663.849336
 5740.36916486 5816.62193452 5892.60876162 5968.33075723 6043.7890268

Costs j:  [ 143.5290351   321.94055867  499.67202908  676.72647222  853.10689795
 1028.8163002  1203.85765706 1378.23393089 1551.94806841 1725.00300079
 1897.40164378 2069.14689774 2240.24164781 2410.68876395 2580.49110106
 2749.65149907 2918.172783   3086.0577631  3253.30923494 3419.92997943
 3585.92276299 3751.29033761 3916.03544092 4080.1607963  4243.66911297
 4406.56308605 4568.84539668 4730.51871208 4891.58568564 5052.048957
 5211.91115215 5371.17488351 5529.84275    5687.91733711 5845.40121701
 6002.29694863 6158.60707772 6314.33413692 6469.48064588 6624.04911131
 6778.04202705 6931.46187418 7084.31112107 7236.59222345 7388.30762451
 7539.45975496 7690.05103311 7840.08386496 7989.56064421 8138.48375243]
Costs j:  [ 108.9319774   205.38543487  301.4862494   397.23597712  492.63616591
  587.68835551  682.39407753  776.75485551  870.77220496  964.44763344
 1057.78264058 1150.77871813 1243.43735004 1335.76001247 1427.74817385
 1519.40329493 1610.72682884 1701.72022112 1792.38490976 1

 7871.03492913 7939.90225943 8008.53429353 8076.93199117 8145.09630739]
Costs j:  [   55.07826426   291.62977708   527.29070588   762.06500347
   995.95660174  1228.96941178  1461.10732401  1692.37420834
  1922.77391424  2152.31027091  2380.98708738  2608.80815264
  2835.7772358   3061.89808615  3287.17443333  3511.60998742
  3735.20843909  3957.9734597   4179.90870141  4401.01779734
  4621.30436161  4840.77198954  5059.42425773  5277.26472414
  5494.29692826  5710.52439119  5925.95061578  6140.57908668
  6354.41327053  6567.45661602  6779.712554    6991.18449761
  7201.87584237  7411.7899663   7620.93023     7829.29997679
  8036.9025328   8243.74120706  8449.81929161  8655.14006161
  8859.70677546  9063.52267485  9266.59098489  9468.91491423
  9670.49765511  9871.34238351 10071.4522592  10270.83042587
 10469.48001122 10667.40412703 10864.6058693  11061.08831831
 11256.85453871 11451.90757963 11646.25047479 11839.88624253
 12032.81788597 12225.04839307 12416.58073669 12607.41787474
 12

Costs j:  [5.48626207e+01 2.91892927e+04 5.05233457e+04 6.58083268e+04
 7.64358698e+04 8.35089671e+04 8.78992347e+04 9.02930543e+04
 9.12287307e+04 9.11263930e+04 9.03120373e+04 8.90368433e+04
 8.74926768e+04 8.58245186e+04 8.41404154e+04 8.25194363e+04
 8.10180233e+04 7.96750507e+04 7.85158477e+04 7.75553877e+04
 7.68008099e+04 7.62534073e+04 7.59101861e+04 7.57650843e+04
 7.58099185e+04 7.60351142e+04 7.64302647e+04 7.69845543e+04
 7.76870750e+04 7.85270581e+04 7.94940417e+04 8.05779859e+04
 8.17693495e+04 8.30591360e+04 8.44389167e+04 8.59008374e+04
 8.74376117e+04 8.90425056e+04 9.07093156e+04 9.24323431e+04
 9.42063656e+04 9.60266071e+04 9.78887076e+04 9.97886934e+04
 9.77956524e+04 9.98645480e+04 1.01942594e+05 1.04028886e+05
 1.06122606e+05 1.08223018e+05]
Costs j:  [  131.52620013   405.18283898   634.77936482   863.49173907
  1091.32390066  1318.27976769  1544.36323747  1769.57818674
  1993.92847174  2217.41792837  2440.0503723   2661.82959909
  2882.75938432  3102.84348372  3

In [40]:
# A.5 train model using optimal values for hyperparameters on test data, report accuracy and test confusion matrix
print(pd.DataFrame(dict_svm))

      C  learn rate init  max iter     tol learn rate  accuracy  \
0   1.0            0.001       100  0.0001   constant  0.000000   
1   1.0            0.001       100  0.0001   adaptive  0.000000   
2   1.0            0.001       100  0.0100   constant  0.000000   
3   1.0            0.001       100  0.0100   adaptive  0.000000   
4   1.0            0.001        50  0.0001   constant  0.000000   
5   1.0            0.001        50  0.0001   adaptive  0.000000   
6   1.0            0.001        50  0.0100   constant  0.000000   
7   1.0            0.001        50  0.0100   adaptive  0.000000   
8   1.0            0.100       100  0.0001   constant  0.333333   
9   1.0            0.100       100  0.0001   adaptive  0.000000   
10  1.0            0.100       100  0.0100   constant  0.333333   
11  1.0            0.100       100  0.0100   adaptive  0.000000   
12  1.0            0.100        50  0.0001   constant  0.000000   
13  1.0            0.100        50  0.0001   adaptive  0.00000

In [99]:
# A.6 plot the learning curve NOT COMPLETED
X = data.drop(columns='quality')
X = (X - X.mean())/X.std() ## standardized data
Y = data['quality']
model = Linear_Regression()
model.fit(X, Y, learning_rate=.01,lambd=1, regularizer='l2')
model.plot_curve()


AttributeError: 'list' object has no attribute 'drop'

In [100]:
# A.7 plot the decision boundary and show support vectors using “decision_boundary_support_vectors”
# https://github.com/rhasanbd/Support-Vector-Machine-Classifier-Beginners-Survival-Kit/blob/master/Support%20Vector%20Machine-1-Linearly%20Separable%20Data.ipynb
# Draw a scatter plot

# need to figure out what 
svm_clf = []

decision_boundary_support_vectors(svm_clf, X)

AttributeError: 'list' object has no attribute 'coef_'

In [61]:
# A.8 implement early stopping in the "fit" method of the Linear_SVC model, generate early stopping code


# this implementation was integrated into the Linear_SVM model A.1


In [ ]:
# Part B: Principle Component Analysis

# B.9 using matplotlib.pyploy "imread" function read the image as a 2D matrix


In [ ]:
# B.10 implement steps of eigendecomposition based PCA on X



In [ ]:
# B.11 find top k eigenvectors and create eigen vector matrix using top k eigenvectors


In [ ]:
# B.12 finally project mean centered data on the k top eigenvectors


In [ ]:
# B.13 reconstruct data matrix by taking dot product between projected data and transpose of top K eigenvec matrix


In [ ]:
# B.14 compute reconstruction error between mean centered data matrix X and reconstructed data matrix


In [ ]:
# B.15 perform steps 11-14 for k: 10, 30, 50, 100, 500
# for each k reconstruct image & print the value of k and reconstruction error